In [2]:
import random
import itertools
import numpy as np
from helpers import *
import clip
import torch
import sklearn
from sklearn.decomposition import PCA
import pandas as pd


In [2]:
#Define concept and sensitive traits

df = pd.read_csv('datasets/all_jobs.csv')

jobs = list(df[df.columns[0]])
print(jobs)



concepts = jobs
#sensitive = ["White", "Black or African American"]
sensitive = ["Hispanic or Latino", "White", "Black or African American", "American Indian and Alaskan Native", "Asian", "Some other race", "Multiracial", "Male", "Female", "Non-binary", "Some other Gender"]

#k: how many influences to draw
k = 1000

itertools.product(concepts, sensitive)
influence_points = influence_information(random_combinations(concepts, sensitive, k))
print(influence_points)

['Accountants and auditors', 'Actors', 'Actuaries', 'Acupuncturists', 'Adhesive bonding machine operators and tenders', 'Administrative law judges, adjudicators, and hearing officers', 'Administrative services managers', 'Adult basic education, adult secondary education, and english as a second language instructors', 'Advertising and promotions managers', 'Advertising sales agents', 'Aerospace engineering and operations technologists and technicians', 'Aerospace engineers', 'Agents and business managers of artists, performers, and athletes', 'Agricultural engineers', 'Agricultural equipment operators', 'Agricultural inspectors', 'Agricultural sciences teachers, postsecondary', 'Agricultural technicians', 'Agricultural workers, all other', 'Air traffic controllers', 'Aircraft cargo handling supervisors', 'Aircraft mechanics and service technicians', 'Aircraft service attendants', 'Aircraft structure, surfaces, rigging, and systems assemblers', 'Airfield operations specialists', 'Airline

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model, preprocess = clip.load("ViT-B/32", device=device)

concept_t = clip.tokenize(["This is a picture of a" + a for a in concepts]).to(device)
influence_points_t = clip.tokenize([x[2] for x in influence_points]).to(device)

with torch.no_grad():
    concept_features = t_normalize(model.encode_text(concept_t))
    influence_points_features = t_normalize(model.encode_text(influence_points_t))

print

# If we project into a different space, do it here

influence_lookup = dict(zip(concepts, concept_features.cpu()))
#print(influence_lookup)

influences = influence_points_features - torch.stack([influence_lookup[x[0]] for x in influence_points]).to(device)
print(influences)


cuda
tensor([[-0.0432,  0.0196,  0.0094,  ..., -0.0651,  0.0101, -0.0294],
        [ 0.0082, -0.0221, -0.0132,  ...,  0.0212,  0.0074, -0.0194],
        [-0.0400,  0.0401, -0.0055,  ...,  0.0571,  0.0142, -0.0681],
        ...,
        [ 0.0188, -0.0176, -0.0183,  ...,  0.0576,  0.0100,  0.0078],
        [-0.0298, -0.0422, -0.0165,  ...,  0.0452, -0.0003, -0.0010],
        [-0.0080, -0.0134,  0.0088,  ...,  0.0623,  0.0184, -0.0233]],
       device='cuda:0', dtype=torch.float16)


In [4]:
#Set an alpha
alpha = .05

#Learn PCA of influences
pca = PCA(n_components=2)
pca.fit(influences.cpu())
print(pca.explained_variance_ratio_)


[0.12224163 0.06966623]


In [5]:

cpu_influences = influences.cpu()
covering_influence_space(pca, cpu_influences)

n = covering_influece_components_search(PCA, cpu_influences)
print(n)

NameError: name 'components_search' is not defined

In [ ]:
pca = PCA(n_components=124, random_state=123)
pca.fit(cpu_influences)


concept_test = clip.tokenize(["This is a picture of a person running", "This is a picture of a black or african american person running"]).to(device)
with torch.no_grad():
    test_features = t_normalize(model.encode_text(concept_test))

diff = test_features[1] - test_features[0]
print(f"Norm of diff: {diff.norm()}")
print(diff)
print(diff.cpu().numpy().reshape(1, -1))
component = pca.inverse_transform(pca.transform(diff.cpu().numpy().reshape(1, -1)))
print(np.linalg.norm(component))
print(np.dot(component / np.linalg.norm(component), t_normalize(diff.cpu())))
norm_1, norm_2 = t_normalize(test_features[0].cpu() + component), t_normalize(test_features[1].cpu())

print(np.dot(norm_1.cpu().numpy(), norm_2.cpu().numpy()))

norm_1, norm_2 = t_normalize(test_features[0].cpu()), t_normalize(test_features[1].cpu())

print(np.dot(norm_1.cpu().numpy(), norm_2.cpu().numpy()))



Norm of diff: 0.411865234375
tensor([ 3.0136e-03, -6.6261e-03, -1.1108e-02,  2.1088e-02,  1.3397e-02,
        -2.7344e-02,  1.8570e-02, -1.2390e-02,  4.4189e-02,  2.8473e-02,
        -1.4362e-03, -6.6833e-03, -3.8757e-02,  2.7069e-02, -3.3264e-03,
        -1.0559e-02,  1.4626e-02,  7.4768e-04, -7.8735e-03,  2.2247e-02,
        -2.9388e-02, -3.0487e-02,  3.0823e-03,  8.5449e-04, -8.4152e-03,
         1.4282e-02, -1.7944e-02, -2.3605e-02,  1.1932e-02,  5.6076e-03,
        -5.6000e-03, -1.4191e-03,  8.5373e-03,  6.4392e-03,  2.0828e-02,
         8.8196e-03,  1.8341e-02,  1.6815e-02, -1.6953e-02,  2.3956e-03,
         5.3711e-03, -1.2558e-02, -7.2250e-03, -4.9744e-02, -1.1902e-03,
        -1.3123e-03, -2.5406e-02, -2.2202e-03, -2.9846e-02,  2.1225e-02,
         4.5509e-03,  2.8061e-02, -9.8953e-03, -8.8196e-03,  1.2840e-02,
        -4.4861e-03,  3.6011e-03,  3.5172e-03,  3.8269e-02,  2.3651e-03,
        -3.2623e-02, -2.9755e-04, -1.9989e-02,  1.2817e-03, -2.5635e-03,
         1.3573e-02,  

In [10]:
classifier = ZeroShotClassifier(['a man', 'a woman'])

In [14]:
from PIL import Image

x = Image.open("CLIP.png")
print(x)
image = [Image.open("CLIP.png")]
text = ["a doctor", "a nurse"]

classifier.predict_proba(text)

<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=2162x762 at 0x21AB0277730>


tensor([[0.7983, 0.2018],
        [0.1329, 0.8672]], device='cuda:0', dtype=torch.float16)

tensor([[[[-1.7923, -1.7923, -1.7923,  ..., -1.7923, -1.7923, -1.7923],
          [-1.7923, -1.7923, -1.7923,  ..., -1.7923, -1.7923, -1.7923],
          [-1.7923, -1.7923, -1.7923,  ..., -1.7923, -1.7923, -1.7923],
          ...,
          [ 1.9303,  1.9303,  1.9303,  ..., -1.7923, -1.7923, -1.7923],
          [ 1.9303,  1.9303,  1.9303,  ..., -1.7923, -1.7923, -1.7923],
          [ 1.0982,  1.0982,  1.0982,  ..., -1.7923, -1.7923, -1.7923]],

         [[-1.7521, -1.7521, -1.7521,  ..., -1.7521, -1.7521, -1.7521],
          [-1.7521, -1.7521, -1.7521,  ..., -1.7521, -1.7521, -1.7521],
          [-1.7521, -1.7521, -1.7521,  ..., -1.7521, -1.7521, -1.7521],
          ...,
          [ 2.0749,  2.0749,  2.0749,  ..., -1.7521, -1.7521, -1.7521],
          [ 2.0749,  2.0749,  2.0749,  ..., -1.7521, -1.7521, -1.7521],
          [ 1.2194,  1.2194,  1.2194,  ..., -1.7521, -1.7521, -1.7521]],

         [[-1.4802, -1.4802, -1.4802,  ..., -1.4802, -1.4802, -1.4802],
          [-1.4802, -1.4802, -

tensor([[ 3.1519e-01, -1.4563e-01,  3.0298e-01, -1.9971e-01,  1.5137e-02,
         -1.0278e-01,  1.5576e-01,  2.9443e-01,  4.8047e-01, -7.2441e-03,
         -3.1567e-01,  1.2805e-01, -1.3843e-01,  5.4736e-01,  2.1350e-01,
         -3.0371e-01, -1.5527e-01,  5.4053e-01, -4.2310e-01,  1.9409e-01,
          2.2781e-02, -1.2372e-01,  1.5198e-01,  8.2581e-02, -5.9326e-01,
          5.9131e-01, -1.4392e-01,  2.0911e-01,  1.6718e-03, -2.7930e-01,
          2.5586e-01,  2.3840e-01, -1.4990e-01,  7.6416e-01,  2.7002e-01,
          2.9099e-02,  1.5649e-01,  4.3018e-01,  1.0681e-02, -2.1367e+00,
         -6.2286e-02,  3.6841e-01, -2.6099e-01, -3.4155e-01, -1.7065e-01,
         -7.1338e-01, -3.4277e-01, -5.0720e-02, -3.2471e-01, -1.1957e-01,
          6.5979e-02, -2.5488e-01, -1.0919e-01,  5.8447e-01,  8.0383e-02,
         -2.9565e-01, -2.2614e-02,  4.7729e-01, -9.6924e-02,  5.6348e-01,
          6.9482e-01, -3.2318e-02,  2.3547e-01, -8.8684e-02,  2.1790e-02,
          6.5063e-02,  2.6270e-01,  7.